# Proyecto: Clasificador de Resultados de Partidos de Fútbol

## Predicción del Ganador de Partidos de Fútbol Utilizando Aprendizaje Automático

Este proyecto tiene como objetivo desarrollar un clasificador capaz de predecir el ganador de un partido de fútbol, informando al modelo el equipo local y el equipo visitante.

Objetivo:
El accuracy del modelo definitivo debe superar el 50% (0.5)

## Carga de Datos y Preparación Inicial


In [ ]:
import os
import pandas as pd
import platform
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Descarga y carga del dataset
if not os.path.exists('partidos.csv'):
    if platform.system() == 'Windows':
        os.system('curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv > partidos.csv')
    else:
        os.system('wget https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv')

df = pd.read_csv('partidos.csv')

# Crear la columna de ganador
# 0: Empate, 1: Gana el local, 2: Gana el visitante
df['winner'] = df.apply(lambda row: 1 if row['home_score'] > row['away_score'] else (2 if row['home_score'] < row['away_score'] else 0), axis=1)
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1


## Preprocesamiento de datos


In [ ]:
# Unificar equipos para encoding
equipos = pd.concat([df['home_team'], df['away_team']]).unique()

# Label Encoding
le = LabelEncoder()
le.fit(equipos)
df_label = df.copy()
df_label['home_team'] = le.transform(df_label['home_team'])
df_label['away_team'] = le.transform(df_label['away_team'])

# One-Hot Encoding
df_onehot = pd.get_dummies(df, columns=['home_team', 'away_team'])

# Binary Encoding
be = BinaryEncoder(cols=['home_team', 'away_team'])
df_binary = be.fit_transform(df)



## Entenamiento y evaluacion


In [ ]:
def train_and_evaluate(data, encoding_type):
    X = data.drop(columns=['winner', 'date', 'tournament', 'city', 'country', 'neutral', 'home_score', 'away_score'])
    y = data['winner']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    best_model = None
    best_accuracy = 0
    best_n_estimators = 0

    for n in [50, 100, 200]:
        model = RandomForestClassifier(n_estimators=n, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f'Accuracy con {n} árboles para {encoding_type}: {accuracy:.4f}')

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_n_estimators = n

    print(f'Mejor modelo para {encoding_type}: {best_n_estimators} árboles con accuracy {best_accuracy:.4f}\n')
    return best_model, best_accuracy


In [ ]:
best_label, acc_label = train_and_evaluate(df_label, 'Label Encoding')
best_onehot, acc_onehot = train_and_evaluate(df_onehot, 'One-Hot Encoding')
best_binary, acc_binary = train_and_evaluate(df_binary, 'Binary Encoding')

Accuracy con 50 árboles para Label Encoding: 0.5014
Accuracy con 100 árboles para Label Encoding: 0.5054
Accuracy con 200 árboles para Label Encoding: 0.5053
Mejor modelo para Label Encoding: 100 árboles con accuracy 0.5054

Accuracy con 50 árboles para One-Hot Encoding: 0.5137
Accuracy con 100 árboles para One-Hot Encoding: 0.5167
Accuracy con 200 árboles para One-Hot Encoding: 0.5174
Mejor modelo para One-Hot Encoding: 200 árboles con accuracy 0.5174

Accuracy con 50 árboles para Binary Encoding: 0.5117
Accuracy con 100 árboles para Binary Encoding: 0.5168
Accuracy con 200 árboles para Binary Encoding: 0.5172
Mejor modelo para Binary Encoding: 200 árboles con accuracy 0.5172

